# fact_messaging_accumulating

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:
mensajeria_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', engineSource)

mensajeria_estadosservicio.drop(columns=["foto", "observaciones", "es_prueba", "foto_binary"], inplace=True)



def llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, dateDW, timeDW):
    dateDB = mensajeria_estadosservicio.iloc[index, 1].strftime("%Y-%m-%d")
    timeDB = mensajeria_estadosservicio.iloc[index, 2].strftime('%H:%M')
    
    biblioteca_de_servicios_y_registros[servicio_id][dateDW] = dateDB
    biblioteca_de_servicios_y_registros[servicio_id][timeDW] = timeDB
        


def llenar_duracion(biblioteca_de_servicios_y_registros, servicio_id, date1, time1, date2, time2, duration):
    datea = biblioteca_de_servicios_y_registros[servicio_id][date1]
    timea = biblioteca_de_servicios_y_registros[servicio_id][time1]

    dateb = biblioteca_de_servicios_y_registros[servicio_id][date2]
    timeb = biblioteca_de_servicios_y_registros[servicio_id][time2]

    if datea and timea and dateb and timeb:
    
        datetimea = datetime.strptime(f"{datea} {timea}", "%Y-%m-%d %H:%M")
        datetimeb = datetime.strptime(f"{dateb} {timeb}", "%Y-%m-%d %H:%M")

        duration_in_minutes = (datetimeb - datetimea).total_seconds() / 60
    
        biblioteca_de_servicios_y_registros[servicio_id][duration] = duration_in_minutes



biblioteca_de_servicios_y_registros = {}

for index, row in mensajeria_estadosservicio.iterrows():
    servicio_id = mensajeria_estadosservicio.iloc[index, 4]
    estado_id = mensajeria_estadosservicio.iloc[index, 3]

    if servicio_id not in biblioteca_de_servicios_y_registros:
        biblioteca_de_servicios_y_registros[servicio_id] = {'key_service': servicio_id, 
                                                            'key_start_date': None, 
                                                            'key_start_time': None, 
                                                            'key_assignment_date': None, 
                                                            'key_assignment_time': None, 
                                                            'key_pick_up_date': None, 
                                                            'key_pick_up_time': None, 
                                                            'key_delivery_date': None, 
                                                            'key_delivery_time': None, 
                                                            'key_closing_date': None, 
                                                            'key_closing_time': None,
                                                            'start_to_assignment_duration': None,
                                                            'assignment_to_pick_up_duration': None,
                                                            'pick_up_to_delivery_duration': None,
                                                            'delivery_to_closing_duration': None,}
    
    if estado_id == 1:
        llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_start_date', 'key_start_time')

    elif estado_id == 2:
        llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_assignment_date', 'key_assignment_time')

    elif estado_id == 4:
        llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_pick_up_date', 'key_pick_up_time')

    elif estado_id == 5:
        llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_delivery_date', 'key_delivery_time')

    elif estado_id == 6:
        llenar_fecha_y_hora(mensajeria_estadosservicio, biblioteca_de_servicios_y_registros, servicio_id, index, 'key_closing_date', 'key_closing_time')




for servicio_id in biblioteca_de_servicios_y_registros:

    llenar_duracion(biblioteca_de_servicios_y_registros, servicio_id, 'key_start_date', 'key_start_time', 'key_assignment_date', 'key_assignment_time', 'start_to_assignment_duration')
    llenar_duracion(biblioteca_de_servicios_y_registros, servicio_id, 'key_assignment_date', 'key_assignment_time', 'key_pick_up_date', 'key_pick_up_time', 'assignment_to_pick_up_duration')
    llenar_duracion(biblioteca_de_servicios_y_registros, servicio_id, 'key_pick_up_date', 'key_pick_up_time', 'key_delivery_date', 'key_delivery_time', 'pick_up_to_delivery_duration')
    llenar_duracion(biblioteca_de_servicios_y_registros, servicio_id, 'key_delivery_date', 'key_delivery_time', 'key_closing_date', 'key_closing_time', 'delivery_to_closing_duration')



fact_messaging = pd.DataFrame.from_dict(biblioteca_de_servicios_y_registros, orient='index')



fact_messaging['key_start_date'] = pd.to_datetime(fact_messaging['key_start_date'], errors='coerce')
fact_messaging['key_assignment_date'] = pd.to_datetime(fact_messaging['key_assignment_date'], errors='coerce')
fact_messaging['key_pick_up_date'] = pd.to_datetime(fact_messaging['key_pick_up_date'], errors='coerce')
fact_messaging['key_delivery_date'] = pd.to_datetime(fact_messaging['key_delivery_date'], errors='coerce')
fact_messaging['key_closing_date'] = pd.to_datetime(fact_messaging['key_closing_date'], errors='coerce')

fact_messaging['key_start_time'] = pd.to_datetime(fact_messaging['key_start_time'], format='%H:%M', errors='coerce').dt.time
fact_messaging['key_assignment_time'] = pd.to_datetime(fact_messaging['key_assignment_time'], format='%H:%M', errors='coerce').dt.time
fact_messaging['key_pick_up_time'] = pd.to_datetime(fact_messaging['key_pick_up_time'], format='%H:%M', errors='coerce').dt.time
fact_messaging['key_delivery_time'] = pd.to_datetime(fact_messaging['key_delivery_time'], format='%H:%M', errors='coerce').dt.time
fact_messaging['key_closing_time'] = pd.to_datetime(fact_messaging['key_closing_time'], format='%H:%M', errors='coerce').dt.time



In [ ]:
fact_messaging['start_to_assignment_duration'].fillna(round(fact_messaging['start_to_assignment_duration'].mean(), 0), inplace=True)
fact_messaging['assignment_to_pick_up_duration'].fillna(round(fact_messaging['assignment_to_pick_up_duration'].mean(), 0), inplace=True)
fact_messaging['pick_up_to_delivery_duration'].fillna(round(fact_messaging['pick_up_to_delivery_duration'].mean(), 0), inplace=True)
fact_messaging['delivery_to_closing_duration'].fillna(round(fact_messaging['delivery_to_closing_duration'].mean(), 0), inplace=True)

from datetime import time

def llenar_fecha_y_hora2(fact_messaging, index, duration_base, date_to_modify, date_base, time_to_modify, time_base):
    duration = fact_messaging.loc[index, duration_base]

    days_d = duration // 1440
    minutes_d = duration % 1440

    if days_d == 0:
        fact_messaging.loc[index, date_to_modify] = fact_messaging.loc[index, date_base]

        time_at_base = fact_messaging.loc[index, time_base]
        
        time_at_modify = timedelta(hours=time_at_base.hour, minutes=time_at_base.minute) + timedelta(minutes=minutes_d)

        if isinstance(time_at_modify, timedelta):
            total_seconds = int(time_at_modify.total_seconds())
            hours = (total_seconds // 3600) % 24 
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            
            time_at_modify = time(hour=hours, minute=minutes, second=seconds)

        fact_messaging.loc[index, time_to_modify] = time_at_modify



for index, row in fact_messaging.iterrows():
    for column, value in row.items():
        if pd.isna(value):
            if column == "key_start_date":
                duration = fact_messaging.loc[index, 'start_to_assignment_duration']

                days_d = duration // 1440
                minutes_d = duration % 1440

                if days_d == 0:
                    fact_messaging.loc[index, 'key_start_date'] = fact_messaging.loc[index, 'key_assignment_date']

                    time_at_assignment = fact_messaging.loc[index, 'key_assignment_time']
                    
                    time_at_start = (timedelta(hours=time_at_assignment.hour, minutes=time_at_assignment.minute) - timedelta(minutes=minutes_d))

                    if isinstance(time_at_start, timedelta):
                        total_seconds = int(time_at_start.total_seconds())
                        hours = (total_seconds // 3600) % 24
                        minutes = (total_seconds % 3600) // 60
                        seconds = total_seconds % 60
                        
                        time_at_start = time(hour=hours, minute=minutes, second=seconds)

                    fact_messaging.loc[index, 'key_start_time'] = time_at_start

                
            if column == "key_assignment_date":
                llenar_fecha_y_hora2(fact_messaging, index, 'start_to_assignment_duration', 'key_assignment_date', 'key_start_date', 'key_assignment_time', 'key_start_time')
            
            if column == "key_assignment_time":
                duration = fact_messaging.loc[index, 'start_to_assignment_duration']
                minutes_d = duration % 1440

                time_at_start = fact_messaging.loc[index, 'key_start_time']
                
                time_at_assignment = (timedelta(hours=time_at_start.hour, minutes=time_at_start.minute) + timedelta(minutes=minutes_d))


                if isinstance(time_at_assignment, timedelta):
                    total_seconds = int(time_at_assignment.total_seconds())
                    hours = (total_seconds // 3600) % 24
                    minutes = (total_seconds % 3600) // 60
                    seconds = total_seconds % 60
                    
                    time_at_assignment = time(hour=hours, minute=minutes, second=seconds)

                fact_messaging.loc[index, 'key_start_time'] = time_at_start

                fact_messaging.loc[index, 'key_assignment_time'] = time_at_assignment
                
            if column == "key_pick_up_date":
                llenar_fecha_y_hora2(fact_messaging, index, 'assignment_to_pick_up_duration', 'key_pick_up_date', 'key_assignment_date', 'key_pick_up_time', 'key_assignment_time') 

            if column == "key_delivery_date":
                llenar_fecha_y_hora2(fact_messaging, index, 'pick_up_to_delivery_duration', 'key_delivery_date', 'key_pick_up_date', 'key_delivery_time', 'key_pick_up_time')
            
            if column == "key_closing_date":
                llenar_fecha_y_hora2(fact_messaging, index, 'delivery_to_closing_duration', 'key_closing_date', 'key_delivery_date', 'key_closing_time', 'key_delivery_time')
            
            

dim_date = pd.read_sql_table('dim_date', engineDestination)
dim_time = pd.read_sql_table('dim_time', engineDestination)


for date_col in ['key_start_date', 'key_assignment_date', 'key_pick_up_date', 'key_delivery_date', 'key_closing_date']:
    fact_messaging = fact_messaging.merge(
        dim_date[['date', 'key_dim_date']], 
        left_on=date_col, 
        right_on='date', 
        how='left'
    ).drop(columns=[date_col]).rename(columns={'key_dim_date': date_col}).drop(columns=['date'])
    fact_messaging[date_col] = pd.to_numeric(fact_messaging[date_col], errors='coerce').astype('Int64')


for time_col in ['key_start_time', 'key_assignment_time', 'key_pick_up_time', 'key_delivery_time', 'key_closing_time']:
    fact_messaging = fact_messaging.merge(
        dim_time[['time', 'key_dim_time']], 
        left_on=time_col, 
        right_on='time', 
        how='left'
    ).drop(columns=[time_col]).rename(columns={'key_dim_time': time_col}).drop(columns=['time'])
    fact_messaging[time_col] = pd.to_numeric(fact_messaging[time_col], errors='coerce').astype('Int64')


fact_messaging["key_fact_messaging_accumulating"] = range(1, len(fact_messaging) + 1)

fact_messaging

In [ ]:
from datetime import date
fact_messaging['saved'] = date.today()

### Carga de datos

In [ ]:
fact_messaging.to_sql("fact_messaging_accumulating", engineDestination, index=False, if_exists="replace")